## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-10-05-26 +0000,nyt,Canada Launched Major Gun Reforms in 2020 Afte...,https://www.nytimes.com/2026/02/10/world/canad...
1,2026-02-11-10-05-12 +0000,nyt,9 Killed and 25 Injured in Shootings at School...,https://www.nytimes.com/2026/02/10/world/canad...
2,2026-02-11-10-04-45 +0000,nyt,Bondi Expected to Face Scrutiny Over Release o...,https://www.nytimes.com/2026/02/11/us/politics...
3,2026-02-11-10-04-45 +0000,nyt,Iran’s Missile Program Tops Israel’s Concerns ...,https://www.nytimes.com/2026/02/11/us/politics...
4,2026-02-11-10-04-39 +0000,nyt,Revelations in Epstein Files Lead to Resignati...,https://www.nytimes.com/2026/02/11/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2493/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,48
19,epstein,21
67,new,19
0,canada,16
199,guthrie,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
89,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,157
229,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,104
86,2026-02-11-03-51-40 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,101
154,2026-02-10-23-53-00 +0000,wsj,Canada’s Carney Tries to Resolve Trump Demands...,https://www.wsj.com/world/americas/carney-look...,99
129,2026-02-11-01-14-19 +0000,latimes,L.A. County officials push new sales tax to of...,https://www.latimes.com/california/story/2026-...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
89,157,2026-02-11-03-37-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
103,80,2026-02-11-02-45-29 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
70,57,2026-02-11-05-11-08 +0000,nypost,Joe Rogan blasts former podcast guest who trie...,https://nypost.com/2026/02/11/us-news/joe-roga...
41,49,2026-02-11-09-03-03 +0000,nypost,Tumbler Ridge students barricaded themselves i...,https://nypost.com/2026/02/11/world-news/tumbl...
327,45,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
174,41,2026-02-10-23-03-05 +0000,latimes,$600 million in Trump administration health cu...,https://www.latimes.com/politics/story/2026-02...
330,40,2026-02-10-16-07-54 +0000,nypost,Nervous Netanyahu pushes up trip to DC for hig...,https://nypost.com/2026/02/10/us-news/nervous-...
358,37,2026-02-10-13-00-00 +0000,latimes,He worked for and romanced Arcadia councilwoma...,https://www.latimes.com/california/story/2026-...
225,36,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...
140,35,2026-02-11-00-21-00 +0000,wsj,Estée Lauder Files Lawsuit Against Walmart All...,https://www.wsj.com/business/retail/estee-laud...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
